In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Connect to OpenAI (use API KEY)

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.

## Define class Website to crawl contents from webpage with given URL
- Use `requests` to send HTTP requests and retrieve webpage content in Python. It allows you to fetch HTML pages and interact with web APIs. Instead of `requests`, we can use `selenium` to handle JavaScripts-rendered contents.
- Use `BeautifulSoup` to parse HTML and XML documents. It helps extract and navigate data from web pages.
  

In [4]:
!which chromedriver

/opt/homebrew/bin/chromedriver


In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument(f"user-agent={headers['User-Agent']}")
        
        service = Service("/opt/homebrew/bin/chromedriver")  # Update with actual path
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            
            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = "No content found"
        finally:
            driver.quit()

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt 
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown in Vietnamese."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [8]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

## And now let's build useful messages for GPT-4o-mini, using a function

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [10]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [11]:
summarize("https://edwarddonner.com")

'## Tóm tắt trang web Edward Donner\n\nTrang web của Edward Donner cung cấp thông tin về cá nhân và công việc của ông. Edward là một lập trình viên và chuyên gia trong lĩnh vực trí tuệ nhân tạo (AI), đồng thời là đồng sáng lập và Giám đốc Công nghệ (CTO) của Nebula.io, nơi ứng dụng AI để giúp người dùng khám phá tiềm năng của họ. Ông cũng là người sáng lập và CEO của công ty khởi nghiệp AI untapt, đã được mua lại vào năm 2021. \n\nTrang web còn giới thiệu một số sự kiện và bài viết liên quan đến trí tuệ nhân tạo, đặc biệt là về các mô hình ngôn ngữ lớn (LLMs). Một số thông tin cập nhật gần đây bao gồm:\n\n- **23 tháng 1, 2025**: Hội thảo về LLM – Tài nguyên thực hành với các đại lý.\n- **21 tháng 12, 2024**: Chào mừng, SuperDataScientists!\n- **13 tháng 11, 2024**: Tài nguyên cho việc nâng cao kỹ năng AI và kỹ thuật LLM.\n- **16 tháng 10, 2024**: Tài nguyên chuyển từ Kỹ sư Phần mềm sang Nhà khoa học Dữ liệu AI.\n\nTrang web khuyến khích kết nối và tương tác với Edward, cũng như cung cấ

In [14]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [15]:
display_summary("https://edwarddonner.com")

# Tóm tắt trang web Edward Donner

Trang web của Edward Donner giới thiệu về cá nhân Ed, một lập trình viên và nhà thử nghiệm LLM (Large Language Model). Ed là đồng sáng lập và CTO của Nebula.io, một công ty ứng dụng trí tuệ nhân tạo để giúp mọi người khám phá tiềm năng của họ trong lĩnh vực tuyển dụng. Ông đã từng là người sáng lập và CEO của startup AI untapt, được mua lại vào năm 2021.

Trang web cũng có một số bài viết và resources liên quan đến LLM, bao gồm:

- **LLM Workshop – Hands-on with Agents – resources** (23 tháng 1, 2025)
- **Welcome, SuperDataScientists!** (21 tháng 12, 2024)
- **Mastering AI and LLM Engineering – Resources** (13 tháng 11, 2024)
- **From Software Engineer to AI Data Scientist – resources** (16 tháng 10, 2024)

Ed cũng chia sẻ sở thích của mình về DJing và sản xuất nhạc điện tử nghiệp dư, và Mời mọi người kết nối để cùng chia sẻ đam mê.

In [16]:
display_summary("https://www.tradingview.com/markets/stocks-usa/market-movers-gainers/")


# Tóm tắt trang web "Top Gaining US Stocks - TradingView"

Trang web "Top Gaining US Stocks" của TradingView cung cấp thông tin về những cổ phiếu Mỹ tăng giá mạnh nhất trong ngày. Danh sách này bao gồm các công ty có tỷ lệ tăng trưởng giá cổ phiếu lớn nhất, cho phép người dùng theo dõi các cơ hội đầu tư tiềm năng.

## Nội dung chính:
- **Danh sách cổ phiếu tăng giá hàng đầu**: Cung cấp thông tin chi tiết về từng cổ phiếu, bao gồm phần trăm thay đổi, giá hiện tại, khối lượng giao dịch, vốn hóa thị trường, và tỷ lệ P/E.
- **Các chỉ số tài chính**: Thông tin về doanh thu, lợi nhuận, và các chỉ số tài chính khác của các công ty.
- **Phân loại cổ phiếu theo loại**: Bao gồm cổ phiếu có vốn hóa lớn, nhỏ, cổ tức cao, và các tỷ số khác như lợi nhuận trên mỗi nhân viên, doanh thu cao nhất, v.v.

## Tin tức và thông báo:
Trang web không cung cấp thông tin tin tức hoặc thông báo cụ thể nào nhưng nhấn mạnh việc người dùng cần cân nhắc các số liệu thống kê trước khi đầu tư, do có rủi ro hồi phục giá cổ phiếu.

### Các công ty và hiệu suất nổi bật:
- **ORGO** (Organogenesis Holdings Inc.): Tăng 102.28%
- **TRNR** (Interactive Strength Inc.): Tăng 67.97%
- **OMI** (Owens & Minor, Inc.): Tăng 39.04%

Website này phục vụ cho các nhà đầu tư muốn theo dõi và phân tích các xu hướng thị trường chứng khoán Mỹ.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)